## Step 5: Basic flooding analysis
create a graph for visualization where disrupted edges that intersect simulated flooded areas are removed from the graph

In [5]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
import osmnx as ox
import pandas as pd
import geopandas as gpd
import networkx as nx
import numpy as np

In [7]:
from shapely.geometry import Point, shape
from shapely import geometry
import shapely

In [8]:
# Get reference to GOSTNets
import sys
sys.path.append(r'C:\repos\GOSTnets')
import GOSTnets as gn

### Fathom SSBN data was used as an input for the simulated flooded areas. The FD_1in20 layer was used, and in QGIS the raster calculater was used to filter raster pixels with a value greater than .25 meters, using this query:
### ("FD_1in20@1" >= 0.25) * 1"
### Then raster was polygonized using QGIS. Finally, the QGIS fix geometries tool was run.

In [9]:
# read back your graph from step 2 from your saved pickle
G = nx.read_gpickle(r"temp\cap_haitien_walk_w_ferries_via_osmnx_w_time.pickle")

In [10]:
import fiona
fc = fiona.open(r"input_folder\FD_1in20_classified_polygonized3.shp")

print(fc.schema)

#first feature of the shapefile
first = fc.next()
print(first) # (GeoJSON format)

{'properties': OrderedDict([('fid', 'float:20'), ('DN', 'int:10')]), 'geometry': 'Polygon'}
{'type': 'Feature', 'id': '0', 'properties': OrderedDict([('fid', 170.0), ('DN', 1)]), 'geometry': {'type': 'Polygon', 'coordinates': [[(-72.4679166665885, 19.822083333666665), (-72.467083333255, 19.822083333666665), (-72.467083333255, 19.821250000333333), (-72.4679166665885, 19.821250000333333), (-72.4679166665885, 19.822083333666665)]]}}


C:\Users\war-machine\Anaconda2\envs\gostnets2\lib\site-packages\ipykernel_launcher.py:7: FionaDeprecationWarning: Collection.__next__() is buggy and will be removed in Fiona 2.0. Switch to `next(iter(collection))`.
  import sys


In [11]:
shp_geom = shape(first['geometry']) # or shp_geom = shape(first) with PyShp)
print(shp_geom)
print(type(shp_geom))

POLYGON ((-72.46791666658849 19.82208333366667, -72.467083333255 19.82208333366667, -72.467083333255 19.82125000033333, -72.46791666658849 19.82125000033333, -72.46791666658849 19.82208333366667))
<class 'shapely.geometry.polygon.Polygon'>


In [12]:
edges_gdf = ox.utils_graph.graph_to_gdfs(G, nodes=False, edges=True, fill_edge_geometry=True)

In [13]:
edges_gdf[:3]

,osmid,highway,oneway,length,geometry,ref,name,maxspeed,service,lanes,bridge,junction,access,ferry,est_width,width,u,v,key
0,"[379649269, 233072790]",footway,False,1227.692,"LINESTRING (-71.99988 19.70057, -72.00021 19.7...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,247844908,3806054858,0
1,"[227769624, 226616997]","[unclassified, footway]",False,1322.904,"LINESTRING (-71.96271 19.56871, -71.96121 19.5...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,330521785,2353905402,0
2,"[227769628, 219143021]",unclassified,False,3659.690,"LINESTRING (-71.96271 19.56871, -71.96282 19.5...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,330521785,330540551,0


### Find intersections between edges and flood polygons

In [14]:
intersection_list = []
#loop through each edge:
for index, row in edges_gdf.iterrows():
    #print(row['geometry'])
    line_geom = row['geometry']
    #loop through each polygon:
    for shapely_poly in fc:
        #print(shapely_poly)
        shapely_poly = shape(shapely_poly['geometry'])
        intersection_line = shapely_poly.intersection(line_geom)
        if intersection_line:
            #print("intersection found")
            #print(intersection_line)
            #print("osmid")
            #print(row['osmid'])
            #if true, add edge to a list of intersections
            intersection_list.append((row['u'],row['v']))

In [15]:
len(intersection_list)

1542

In [16]:
# get rid of duplicates
intersection_list = list(set(intersection_list))

In [17]:
len(intersection_list)

1492

In [18]:
print(nx.info(G))

Name: 
Type: MultiDiGraph
Number of nodes: 12931
Number of edges: 34334
Average in degree:   2.6552
Average out degree:   2.6552


In [19]:
G_flooded = G.copy()

### need to remove disrupted edges from the graph
We need to remove the edges from the graph that intersect the flooded area. Then using only the walk/ferry graph we can run accessibility analysis and see how many routes are affected.

In [20]:
for line in intersection_list:
    G_flooded.remove_edge(line[0],line[1])

In [21]:
print(nx.info(G_flooded))

Name: 
Type: MultiDiGraph
Number of nodes: 12931
Number of edges: 32842
Average in degree:   2.5398
Average out degree:   2.5398


### need to take the largest connected graph and also lose some potentially dangling nodes 

In [22]:
# compatible with NetworkX 2.4
list_of_subgraphs = list(G_flooded.subgraph(c).copy() for c in nx.strongly_connected_components(G_flooded))
max_graph = None
max_edges = 0

# To sort the list in place...
list_of_subgraphs.sort(key=lambda x: x.number_of_edges(), reverse=True)

# for i in list_of_subgraphs:
#     print(f"The {i} graph contains {i.number_of_edges()} edges")
#     if i.number_of_edges() > max_edges:
#         max_edges = i.number_of_edges()
#         max_graph = i

# set your graph equal to the largest sub-graph
G_largest_flooded = list_of_subgraphs[0]

The  graph contains <bound method MultiGraph.number_of_edges of <networkx.classes.multidigraph.MultiDiGraph object at 0x000001C43AC38048>> edges
The  graph contains <bound method MultiGraph.number_of_edges of <networkx.classes.multidigraph.MultiDiGraph object at 0x000001C43AC3E4C8>> edges
The  graph contains <bound method MultiGraph.number_of_edges of <networkx.classes.multidigraph.MultiDiGraph object at 0x000001C43AC3E5C8>> edges
The  graph contains <bound method MultiGraph.number_of_edges of <networkx.classes.multidigraph.MultiDiGraph object at 0x000001C43AC3E648>> edges
The  graph contains <bound method MultiGraph.number_of_edges of <networkx.classes.multidigraph.MultiDiGraph object at 0x000001C43AC3E488>> edges
The  graph contains <bound method MultiGraph.number_of_edges of <networkx.classes.multidigraph.MultiDiGraph object at 0x000001C43AC3E6C8>> edges
The  graph contains <bound method MultiGraph.number_of_edges of <networkx.classes.multidigraph.MultiDiGraph object at 0x000001C43A

In [25]:
print(nx.info(G_largest_flooded))

Name: 
Type: MultiDiGraph
Number of nodes: 4853
Number of edges: 13060
Average in degree:   2.6911
Average out degree:   2.6911


In [28]:
G_largest_flooded.number_of_edges()

13060

In [31]:
len(list_of_subgraphs)

504

In [37]:
print(list_of_subgraphs[0].number_of_edges())

13060


In [38]:
print(print(list_of_subgraphs[1].number_of_edges())

7156


In [39]:
print(list_of_subgraphs[2].number_of_edges())

6048


In [40]:
print(list_of_subgraphs[3].number_of_edges())

4936


In [41]:
print(list_of_subgraphs[4].number_of_edges())

374


### For visualization purposes save the top 3 largest disconnected graphs

In [27]:
gn.save(G_largest_flooded,"cap_haitien_walk_w_ferries_flooded_largest_1", "temp",pickle = False, edges = True, nodes = False)

In [42]:
gn.save(list_of_subgraphs[1],"cap_haitien_walk_w_ferries_flooded_largest_2", "temp",pickle = False, edges = True, nodes = False)

In [43]:
gn.save(list_of_subgraphs[2],"cap_haitien_walk_w_ferries_flooded_largest_3", "temp",pickle = False, edges = True, nodes = False)

In [44]:
gn.save(list_of_subgraphs[3],"cap_haitien_walk_w_ferries_flooded_largest_4", "temp",pickle = False, edges = True, nodes = False)